In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mfcc.csv to mfcc.csv


In [ ]:
import io
import pandas as pd

mfcc = pd.read_csv(io.StringIO(uploaded['mfcc.csv'].decode('utf-8')))
mfcc.drop(['Unnamed: 0'], axis = 1, inplace = True)
mfcc = mfcc.reset_index(drop=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving seol.csv to seol.csv


In [ ]:
import io
import pandas as pd

seol = pd.read_csv(io.StringIO(uploaded['seol.csv'].decode('utf-8')))
di = seol.iloc[:,3]
seol = seol.drop(['Diagnosis'],axis =1)
seol.drop(['Unnamed: 0'], axis = 1, inplace = True)
seol = seol.reset_index(drop=True)

In [ ]:
import math 
import numpy as np
import pandas as pd

In [ ]:
!pip install boruta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
# 선택된 변수들 추출
selected_features = seol.columns[boruta.support_]

# 선택된 변수들 출력
print(selected_features)

Index(['Age', 'Voice Handicap Index (VHI) Score',
       'Reflux Symptom Index (RSI) Score',
       'Number of cups of coffee drinked in a day'],
      dtype='object')


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import KFold
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

본 코드

In [ ]:

kf = KFold(n_splits=5)
# 랜덤 포레스트 모델 생성
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

a = seol.values
b = di.values
X = mfcc.values
# 데이터 정규화
sc = StandardScaler()
X = sc.fit_transform(X)
X = X.reshape(X.shape[0], -1)

# Boruta 알고리즘으로 변수 선택
boruta = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42,perc = 60)
boruta.fit(X,b)

# 선택된 변수만 추출
selected_X = X[:, boruta.support_]

# 학습 데이터와 테스트 데이터로 분할

# 전체 데이터셋 인덱스 생성

random_state = np.random.RandomState(seed=42)
best_params_list = []
best_score_list = []
xgbs = []
mlps = []
for train_index, test_index in kf.split(a):
    X_train, X_test = a[train_index], a[test_index]
    y_train, y_test = b[train_index], b[test_index]

    xgb =XGBClassifier(n_estimators=100,max_depth =2,subsample=1,
            reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.005,colsample_bytree = 1,min_child_weight=2,gamma = 1)
    # 모델 예측
    fit = xgb.fit(X_train, y_train)
    xgb_preds = fit.predict(X_test)
    for element in xgb_preds:
      xgbs.append(element)

    X_train, X_test = selected_X[train_index], selected_X[test_index]
    y_train, y_test = b[train_index], b[test_index] 

    mlp = MLPClassifier(hidden_layer_sizes=(8,8,8,8,8), activation='relu', solver='adam', alpha=0.01, batch_size=22,
                        learning_rate='constant', learning_rate_init=0.04, power_t=0.5, max_iter=150, random_state=42)

    # 모델 학습
    mlp.fit(X_train, y_train)
    # 예측
    y_pred = mlp.predict(X_test)
    # 모델 예측
    mlp_preds = mlp.predict(X_test)
    for element in mlp_preds:
      mlps.append(element)
 
df_pred = pd.DataFrame({'xgb_pred': xgbs, 'mlps': mlps})

rf = RandomForestClassifier(n_estimators=150, max_depth =2,random_state=42)
rf.fit(df_pred, b)

final_preds = rf.predict(df_pred)
acc = accuracy_score(b, final_preds)
print('Accuracy:', acc)

Accuracy: 0.7932692307692307


In [ ]:
cm = confusion_matrix(b, final_preds)
cm2 = confusion_matrix(b, xgbs)
cm3 = confusion_matrix(b, mlps)
sensitivity = cm[0, 0] / (cm[0, 0] + cm[1, 0])
sensitivity2 = cm2[0, 0] / (cm2[0, 0] + cm2[1, 0])
sensitivity3 = cm3[0, 0] / (cm3[0, 0] + cm3[1, 0])

print("민감도:", sensitivity)
print("민감도:", sensitivity2)
print("민감도:", sensitivity3)

민감도: 0.8103448275862069
민감도: 0.8165680473372781
민감도: 0.7152777777777778


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_resampled, final_preds)

NameError: ignored

In [ ]:
random_state = np.random.RandomState(seed=42)
best_params_list = []
best_score_list = []
xgbs = []
mlps = []
for train_index, test_index in kf.split(a):
    X_train, X_test = a[train_index], a[test_index]
    y_train, y_test = b[train_index], b[test_index]

    xgb =XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
            reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.005,colsample_bytree = 1,min_child_weight=2,gamma = 1)
    # 모델 예측
    fit = xgb.fit(X_train, y_train)
    xgb_preds = fit.predict(X_test)
    for element in xgb_preds:
      xgbs.append(element)

    X_train, X_test = selected_X[train_index], selected_X[test_index]
    y_train, y_test = b[train_index], b[test_index] 

    mlp = MLPClassifier(hidden_layer_sizes=(8,8,8,8,8), activation='relu', solver='adam', alpha=0.01, batch_size=22,
                        learning_rate='constant', learning_rate_init=0.04, power_t=0.5, max_iter=150, random_state=42)

    # 모델 학습
    mlp.fit(X_train, y_train)
    # 예측
    y_pred = mlp.predict(X_test)
    # 모델 예측
    mlp_preds = mlp.predict(X_test)
    for element in mlp_preds:
      mlps.append(element)
 
df_pred = pd.DataFrame({'xgb_pred': xgbs, 'mlps': mlps})

rf = RandomForestClassifier(n_estimators=150, max_depth =2,random_state=42)
rf.fit(df_pred, b)

final_preds = rf.predict(df_pred)
acc = accuracy_score(b, final_preds)
print('Accuracy:', acc)

Accuracy: 0.7548076923076923


smote 사용

In [ ]:
import random
random.seed(42) 

a = seol.values
b = di.values
X = mfcc.values
# 데이터 정규화
sc = StandardScaler()
X = sc.fit_transform(X)
X = X.reshape(X.shape[0], -1)

rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# Boruta 알고리즘으로 변수 선택
boruta = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42,perc = 50)
boruta.fit(X,b)

# 선택된 변수만 추출
selected_X = X[:, boruta.support_]

from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(selected_X, b)
kf = KFold(n_splits=5)
acc_scores = []
mlps = []
proba1 = []
proba2 = []

for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled[train_index], X_resampled[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index] 

        mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
                        learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

        # 모델 학습
        mlp.fit(X_train, y_train)
        # 예측
        y_pred = mlp.predict(X_test)

        # 정확도 및 특이도 계산 및 저장
        acc = accuracy_score(y_test, y_pred)
        acc_scores.append(acc)
        mlp_preds = mlp.predict(X_test)
        for element in mlp_preds:
            mlps.append(element)  
        prob = mlp.predict_proba(X_test) 
        for e in prob:
          proba1.append(e)
          
# 다수의 정확도 및 특이도를 평균 내어 출력
print('Mean accuracy: ', np.mean(acc_scores))

a = seol.values
b = di.values

X_resampled2, y_resampled2 = smote.fit_resample(a, b)

xgbs = []
acc_scores = []
for train_index, test_index in kf.split(X_resampled2):
      X_train, X_test = X_resampled2[train_index], X_resampled2[test_index]
      y_train, y_test = y_resampled2[train_index], y_resampled2[test_index]
      xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
      fit = xgb.fit(X_train, y_train)
      preds1 = fit.predict(X_test)
      acc = accuracy_score(y_test, preds1)
      acc_scores.append(acc)
      for element in preds1:
        xgbs.append(element)
      prob = xgb.predict_proba(X_test) 
      for e in prob:
        proba2.append(e)    
print('Mean accuracy: ', np.mean(acc_scores))

df_pred = pd.DataFrame({'xgb_pred': xgbs, 'mlps': mlps})

rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(df_pred, y_resampled2)

final_preds = rf.predict(df_pred)
acc = accuracy_score(y_resampled2, final_preds)

Mean accuracy:  0.8052459016393442
Mean accuracy:  0.8185792349726775
민감도: 0.8255813953488372
Accuracy: 0.8708609271523179


NameError: ignored

In [ ]:
prob1 = pd.DataFrame(proba1)
prob2 = pd.DataFrame(proba2)
result = pd.concat([prob1,prob2], axis=1)

In [ ]:
rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(result, y_resampled2)

final_preds = rf.predict(result)
acc = accuracy_score(y_resampled2, final_preds)

cm = confusion_matrix(y_resampled, final_preds)
sensitivity = cm[0, 0] / (cm[0, 0] + cm[1, 0])
print("민감도:", sensitivity)
print('Accuracy:', acc)
prec = precision(cm)
print("Precision:", prec)
rec = recall(cm)
print("Recall:", rec)
f1 = f1_score(cm)
print("F1 Score:", f1)

민감도: 0.8614457831325302
Accuracy: 0.8973509933774835
Precision: 0.9411764705882353
Recall: 0.847682119205298
F1 Score: 0.89198606271777


In [ ]:
def calculate_metrics(y_true, y_pred):
    # 혼동 행렬 생성
    cm = confusion_matrix(y_true, y_pred)

    # True Positive, False Positive, False Negative, True Negative 추출
    tn, fp, fn, tp = cm.ravel()

    # 정확도(Accuracy) 계산
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # 정밀도(Precision) 계산
    precision = tp / (tp + fp)

    # 재현율(Recall) 계산
    recall = tp / (tp + fn)

    # 특이도(Specificity) 계산
    specificity = tn / (tn + fp)

    # F1 스코어(F1 score) 계산
    f1_score = 2 * precision * recall / (precision + recall)

    return accuracy, precision, recall, specificity, f1_score

In [ ]:
accuracy, precision, recall, specificity, f1_score = calculate_metrics(y_resampled,mlps)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)
print("F1 Score:", f1_score)

Accuracy: 0.804635761589404
Precision: 0.7584269662921348
Recall: 0.8940397350993378
Specificity: 0.7152317880794702
F1 Score: 0.8206686930091186


In [ ]:
def precision(confusion_matrix):
    tp = confusion_matrix[1, 1]  # True Positive
    fp = confusion_matrix[0, 1]  # False Positive
    return tp / (tp + fp)

def recall(confusion_matrix):
    tp = confusion_matrix[1, 1]  # True Positive
    fn = confusion_matrix[1, 0]  # False Negative
    return tp / (tp + fn)

def f1_score(confusion_matrix):
    prec = precision(confusion_matrix)
    rec = recall(confusion_matrix)
    return 2 * (prec * rec) / (prec + rec)

cm = confusion_matrix(y_resampled, mlps)
prec = precision(cm)
print("Precision:", prec)

# 특이도를 계산합니다.
rec = recall(cm)
print("Recall:", rec)

# F1 스코어를 계산합니다.
f1 = f1_score(cm)
print("F1 Score:", f1)

Precision: 0.7584269662921348
Recall: 0.8940397350993378
F1 Score: 0.8206686930091186


In [ ]:
# 선택된 변수들 추출
selected_features = mfcc.columns[boruta.support_]

# 선택된 변수들 출력
print(selected_features)

Index(['q75', 'q100', 'diff', 'fft_peak', 'mfcc_means_3', 'mfcc_means_8',
       'mfcc_means_12', 'mfcc_std_1', 'mfcc_std_9', 'mfcc_std_10',
       'mfcc_std_11', 'mfcc_std_13', 'mfcc_max_3', 'mfcc_min_1', 'mfcc_min_3',
       'mfcc_min_12'],
      dtype='object')


mlp

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(selected_X, b)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
kf = KFold(n_splits=5)
acc_scores = []
spec_scores = []
mlps = []
probs2 = []

for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled[train_index], X_resampled[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index] 

        mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
                        learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

        # 모델 학습
        mlp.fit(X_train, y_train)
        # 예측
        y_pred = mlp.predict(X_test)

        # 정확도 및 특이도 계산 및 저장
        acc = accuracy_score(y_test, y_pred)
        acc_scores.append(acc)
        mlp_preds = mlp.predict(X_test)
        for element in mlp_preds:
            mlps.append(element)
        mlp_probs = mlp.predict_proba(X_test)
        for e in mlp_probs:
          probs2.append(e)
            
# 다수의 정확도 및 특이도를 평균 내어 출력
print('Mean accuracy: ', np.mean(acc_scores))

Mean accuracy:  0.8052459016393442


xgboost

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled2, y_resampled2 = smote.fit_resample(a, b)

In [ ]:
kf = KFold(n_splits=5)

a = seol.values
b = di.values

xgbs = []
acc_scores = []
recall_scores = []
probs = []
for train_index, test_index in kf.split(X_resampled2):
      X_train, X_test = X_resampled2[train_index], X_resampled2[test_index]
      y_train, y_test = y_resampled2[train_index], y_resampled2[test_index]
      xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
      fit = xgb.fit(X_train, y_train)
      preds1 = fit.predict(X_test)
      acc = accuracy_score(y_test, preds1)
      acc_scores.append(acc)
      for element in preds1:
        xgbs.append(element)
      xgb_probs = xgb.predict_proba(X_test)
      for e in xgb_probs:
        probs.append(e)
print('Mean accuracy: ', np.mean(acc_scores))

Mean accuracy:  0.8185792349726775


In [ ]:
print(len(xgbs),len(mlps))

302 302


In [ ]:
df_pred = pd.DataFrame({'xgb_pred': xgbs, 'mlps': mlps})

rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(df_pred, y_resampled2)

final_preds = rf.predict(df_pred)
acc = accuracy_score(y_resampled2, final_preds)
print('Accuracy:', acc)

Accuracy: 0.8145695364238411


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_resampled2, final_preds)

array([[138,  13],
       [ 49, 102]])

In [ ]:
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# 입력 데이터와 출력 데이터로 분리
X = mfcc.values
y = di.values

# 데이터 정규화
sc = StandardScaler()
X = sc.fit_transform(X)

X = X.reshape(X.shape[0], -1)

# 학습 데이터와 평가 데이터로 분리
kf = KFold(n_splits=10, shuffle=True, random_state=42)
best_params_list = []
best_score_list = []
for train_index, test_index in kf.split(a):
    X_train, X_test = selected_X[train_index], selected_X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    mlp = MLPClassifier(random_state=42)

    # Grid Search
    param_grid = {
        'hidden_layer_sizes': [(8,8,8,8,8),(7,7,7,7,7),(10,10,10,10,10)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.01],
        'batch_size': [20,22,24,26],
        'learning_rate': ['constant'],
        'learning_rate_init': [0.1,0.05,0.01,0.06],
        'power_t': [0.5],
        'max_iter': [100,150,200]
    }

    grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_params_list.append(grid_search.best_params_)
    best_score_list.append(grid_search.best_score_)

# Grid Search 결과 출력

Average best parameters found: 


NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)
# 랜덤 포레스트 모델 생성
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=5, random_state=42)

a = seol.values
b = di.values
X = mfcc.values
# 데이터 정규화
sc = StandardScaler()
X = sc.fit_transform(X)
X = X.reshape(X.shape[0], -1)

# Boruta 알고리즘으로 변수 선택
boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42,perc = 80)
boruta.fit(a,b)

# 선택된 변수만 추출
selected_X = a[:, boruta.support_]

# 학습 데이터와 테스트 데이터로 분할

# 전체 데이터셋 인덱스 생성

random_state = np.random.RandomState(seed=42)
total_indices = np.arange(len(a))
best_params_list = []
best_score_list = []
xgbs = []
acc_scores = []
mlps = []
for train_index, test_index in kf.split(a):
    X_train, X_test = a[train_index], a[test_index]
    y_train, y_test = b[train_index], b[test_index]

    xgb = XGBClassifier()
    # 하이퍼파라미터 그리드 설정
    param_grid = {
        'max_depth': [2, 4, 6, 8],
        'n_estimators': [50, 100, 200, 400],
        'learning_rate': [0.01, 0.001,0.005,0.1]}
    grid_search = GridSearchCV(xgb, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_params_list.append(grid_search.best_params_)
    best_score_list.append(grid_search.best_score_)

    # 모델 예측
    xgb_preds = grid_search.best_estimator_.predict(X_test)
    for element in xgb_preds:
      xgbs.append(element)
print(grid_search.best_score_)

In [ ]:
grid_search.best_params_

{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 200}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)
# 랜덤 포레스트 모델 생성

a = seol.values
b = di.values
X = mfcc.values
# 데이터 정규화
sc = StandardScaler()
X = sc.fit_transform(X)
X = X.reshape(X.shape[0], -1)

# 학습 데이터와 테스트 데이터로 분할

# 전체 데이터셋 인덱스 생성

random_state = np.random.RandomState(seed=42)
xgbs = []
acc_scores = []
mlps = []
for train_index, test_index in kf.split(a):
    X_train, X_test = a[train_index], a[test_index]
    y_train, y_test = b[train_index], b[test_index]

    xgb = XGBClassifier(n_estimators=50,max_depth =19,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.003,colsample_bytree = 1,min_child_weight=2,gamma = 1)
    # 모델 예측
    fit = xgb.fit(X_train, y_train)
    xgb_preds = fit.predict(X_test)
    for element in xgb_preds:
      xgbs.append(element)

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = b[train_index], b[test_index] 

    mlp = MLPClassifier(hidden_layer_sizes=(7,7,7,7,7), activation='relu', solver='adam', alpha=0.01, batch_size=20,
                     learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=160, random_state=42) #그리드 서치가 너무 많이 걸려서 따로 계산함

    # 모델 학습
    mlp.fit(X_train, y_train)
    # 예측
    y_pred = mlp.predict(X_test)

    # 정확도 계산 및 저장
    acc = accuracy_score(y_test, y_pred)
    acc_scores.append(acc)
    # 모델 예측
    mlp_preds = mlp.predict(X_test)
    for element in mlp_preds:
      mlps.append(element)
 
df_pred = pd.DataFrame({'xgb_pred': xgbs, 'mlps': mlps})

rf = RandomForestClassifier(n_estimators=150, max_depth =2,random_state=42)
rf.fit(df_pred, b)

final_preds = rf.predict(df_pred)
acc = accuracy_score(b, final_preds)
print('Accuracy:', acc)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (160) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (160) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (160) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (160) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Accuracy: 0.7451923076923077
